In [ ]:
!pip install ydata-profiling

In [9]:
import pandas as pd
from ydata_profiling import ProfileReport

#### Project Setup and Data Collection: ✅

1) Set up your Jupyter notebook ✅
2) Scrape or pull data from online resources ✅
3) Load the data into your notebook ✅ <br /><br />


In [4]:
df = pd.read_csv('dataset.csv', index_col=0)

2. Exploratory Data Analysis (EDA):
    - 2.1 Initial data inspection (shape, info, describe)  ✅
    - 2.2 Generate and save pandas profiling report  ✅
    - 2.3 Visualize data distributions ('IMDB Rating', 'Budget', 'Gross Worldwide', 'Runtime')
    - 2.4 Explore relationships between 'Gross Worldwide' and other variables
    - 2.5 Analyze categorical data ('Directors', 'Writers', 'Stars', 'Origin Countries', 'Languages', 'Genres')
    - 2.6 Identify potential issues (missing values, duplicates)<br /><br />


In [5]:
# Initial data inspection (shape, info, describe)
df.shape

(46646, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46646 entries, 0 to 46645
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Movie ID          46646 non-null  object 
 1   Title             46573 non-null  object 
 2   IMDB Rating       24971 non-null  float64
 3   Directors         43308 non-null  object 
 4   Writers           43308 non-null  object 
 5   Stars             43308 non-null  object 
 6   Storyline         42363 non-null  object 
 7   Origin Countries  44661 non-null  object 
 8   Languages         43435 non-null  object 
 9   Budget            14748 non-null  object 
 10  Gross Worldwide   17622 non-null  object 
 11  Runtime           40785 non-null  object 
 12  Genres            46572 non-null  object 
dtypes: float64(1), object(12)
memory usage: 5.0+ MB


In [7]:
df.describe()

,IMDB Rating
count,24971.000000
mean,6.135405
std,1.223421
min,1.000000
25%,5.400000
50%,6.300000
75%,7.000000
max,10.000000


In [10]:
profile = ProfileReport(df, title="Movie Report")
profile.to_file('movie_report.html')

/Users/mac/anaconda3/lib/python3.11/site-packages/ydata_profiling/profile_report.py:363: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

3. Data Cleaning and Preprocessing:
    - 3.1 Handle missing values
    - 3.2 Remove duplicates
    - 3.3 Correct data types (ensure 'Budget' and 'Gross Worldwide', 'Runtime', etc. are numerical)
    - 3.4 Handle outliers:
        - 3.4.1 Detect outliers (boxplots and IQR)
        - 3.4.2 Apply treatment methods:
            - Quantile-based Flooring and Capping
            - Trimming
            - Log Transformation
    - 3.5 Compare and discuss outcomes of outlier treatments<br /><br />


4. Feature Engineering:
    - 4.1 Encode categorical variables ('Directors', 'Writers', 'Stars', 'Origin Countries', 'Languages', 'Genres' etc)
    - 4.2 Extract features from 'Storyline' using NLP techniques (optional)
    - 4.3 Create new features if applicable (e.g., director/star popularity, genre combinations) (optional)
    - 4.4 Normalize/standardize numerical features if needed<br /><br />


5. Feature Selection:
    - 5.1 Identify most important features for predicting 'Gross Worldwide' / revenue_usd
    - 5.2 Use correlation analysis or feature importance techniques<br /><br />


6. Natural Language Processing (if applicable):
    - 6.1 Text data cleaning
    - 6.2 Tokenization
    - 6.3 Stop word and punctuation removal
    - 6.4 Stemming or lemmatization<br /><br />


7. Model Selection and Training:
    - 7.1 Split data into training and testing sets
    - 7.2 Select and train multiple regression models (e.g., Linear Regression, Random Forest, Gradient Boosting)
    - 7.3 Perform cross-validation<br /><br />


8. Results and Visualization (based on regression models):
    - 8.1 Create meaningful visualizations of your findings (e.g., actual vs predicted revenues)
    - 8.2 Interpret and explain results<br /><br />


9. Unsupervised Learning:
    - 9.1 Select and apply two unsupervised learning methods
    - 9.2 Analyze and interpret results
    - 9.3 Compare outcomes of different methods
    - 9.4 Create a feature from this cluster generated and put it into the regression model 
    - 9.5 Analyse the result from the new feature you engineered    <br /><br />


10. Results and Visualization (based on unsupervised learning):
    - 10.1 Create meaningful visualizations of your findings (e.g., actual vs predicted revenues)
    - 10.2 Interpret and explain results<br /><br />


11. Conclusion and Recommendations:
    - 11.1 Summarize key findings
    - 11.2 Discuss the most influential factors for movie revenue
    - 11.3 Provide actionable insights or recommendations<br /><br />